In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow
#It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.
#Supports both convolutional networks(Image processing) and recurrent networks(Natural language processing), as well as combinations of the two.



from keras.preprocessing.text import Tokenizer#Used to split data
from keras.preprocessing.sequence import pad_sequences#used to pad data
from keras.models import Sequential#The Sequential model is a linear stack of layers
from keras.layers import Dense, Embedding, LSTM
#A dense layer is just a regular layer of neurons in a neural network. Each neuron recieves input from all the neurons in the previous layer, thus densely connected
#Dropout is a regularization technique, which aims to reduce the complexity of the model with the goal to prevent overfitting.
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
#reading data from file
data = pd.read_csv('Sentiment.csv')

# Keeping only the neccessary columns
data = data[['text','sentiment']]
print(len(data))#number of training data

13871


In [3]:
data = data[data.sentiment != "Neutral"]#remove neutral data sets because it gives no information

In [4]:
print (data)



                                                    text sentiment
1      RT @ScottWalker: Didn't catch the full #GOPdeb...  Positive
3      RT @RobGeorge: That Carly Fiorina is trending ...  Positive
4      RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  Positive
5      RT @GregAbbott_TX: @TedCruz: "On my first day ...  Positive
6      RT @warriorwoman91: I liked her and was happy ...  Negative
8      Deer in the headlights RT @lizzwinstead: Ben C...  Negative
9      RT @NancyOsborne180: Last night's debate prove...  Negative
10     @JGreenDC @realDonaldTrump In all fairness #Bi...  Negative
11     RT @WayneDupreeShow: Just woke up to tweet thi...  Positive
12     Me reading my family's comments about how grea...  Negative
14     RT @pattonoswalt: I loved Scott Walker as Mark...  Positive
15     Hey @ChrisChristie exploiting the tragedy of 9...  Negative
16     RT @CarolCNN: #DonaldTrump under fire for comm...  Negative
17     RT @johncardillo: Guess who had most speaking ...  Nega

In [5]:
data['text'] = data['text'].apply(lambda x: x.lower())#change it all to lowercase to avoid any differences

print(data)



                                                    text sentiment
1      rt @scottwalker: didn't catch the full #gopdeb...  Positive
3      rt @robgeorge: that carly fiorina is trending ...  Positive
4      rt @danscavino: #gopdebate w/ @realdonaldtrump...  Positive
5      rt @gregabbott_tx: @tedcruz: "on my first day ...  Positive
6      rt @warriorwoman91: i liked her and was happy ...  Negative
8      deer in the headlights rt @lizzwinstead: ben c...  Negative
9      rt @nancyosborne180: last night's debate prove...  Negative
10     @jgreendc @realdonaldtrump in all fairness #bi...  Negative
11     rt @waynedupreeshow: just woke up to tweet thi...  Positive
12     me reading my family's comments about how grea...  Negative
14     rt @pattonoswalt: i loved scott walker as mark...  Positive
15     hey @chrischristie exploiting the tragedy of 9...  Negative
16     rt @carolcnn: #donaldtrump under fire for comm...  Negative
17     rt @johncardillo: guess who had most speaking ...  Nega

In [6]:
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))#remove unnecessary expressions like @
print (data)

                                                    text sentiment
1      rt scottwalker didnt catch the full gopdebate ...  Positive
3      rt robgeorge that carly fiorina is trending  h...  Positive
4      rt danscavino gopdebate w realdonaldtrump deli...  Positive
5      rt gregabbott_tx tedcruz on my first day i wil...  Positive
6      rt warriorwoman91 i liked her and was happy wh...  Negative
8      deer in the headlights rt lizzwinstead ben car...  Negative
9      rt nancyosborne180 last nights debate proved i...  Negative
10     jgreendc realdonaldtrump in all fairness billc...  Negative
11     rt waynedupreeshow just woke up to tweet this ...  Positive
12     me reading my familys comments about how great...  Negative
14     rt pattonoswalt i loved scott walker as mark h...  Positive
15     hey chrischristie exploiting the tragedy of 91...  Negative
16     rt carolcnn donaldtrump under fire for comment...  Negative
17     rt johncardillo guess who had most speaking ti...  Nega

In [7]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
print (data)

                                                    text sentiment
1        scottwalker didnt catch the full gopdebate l...  Positive
3        robgeorge that carly fiorina is trending  ho...  Positive
4        danscavino gopdebate w realdonaldtrump deliv...  Positive
5        gregabbott_tx tedcruz on my first day i will...  Positive
6        warriorwoman91 i liked her and was happy whe...  Negative
8      deer in the headlights   lizzwinstead ben cars...  Negative
9        nancyosborne180 last nights debate proved it...  Negative
10     jgreendc realdonaldtrump in all fairness billc...  Negative
11       waynedupreeshow just woke up to tweet this o...  Positive
12     me reading my familys comments about how great...  Negative
14       pattonoswalt i loved scott walker as mark ha...  Positive
15     hey chrischristie exploiting the tragedy of 91...  Negative
16       carolcnn donaldtrump under fire for comments...  Negative
17       johncardillo guess who had most speaking tim...  Nega

In [8]:
batch_size = 5000
tokenizer = Tokenizer(nb_words=batch_size, split=' ')#split as words

tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
#convert the text into stream of numbers so it can be fed into the computer

c:\users\mahalakshumi\myenv\lib\site-packages\keras_preprocessing\text.py:177: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [9]:
for ll in X:
    print(len(ll), ll)

19 [363, 122, 2404, 1, 703, 2, 39, 58, 237, 37, 210, 6, 4277, 174, 1761, 12, 1324, 1409, 743]
19 [16, 284, 252, 5, 4278, 821, 102, 167, 26, 2796, 136, 6, 1, 173, 12, 2, 233, 724, 17]
16 [1261, 2, 303, 23, 1943, 1, 1632, 216, 12, 1, 704, 6, 185, 207, 371, 670]
20 [4279, 2405, 127, 17, 53, 263, 410, 9, 82, 3322, 304, 441, 1325, 1762, 1150, 62, 1944, 194, 2, 51]
22 [2797, 9, 1151, 167, 8, 21, 1326, 63, 9, 614, 188, 21, 189, 4, 34, 1, 563, 19, 822, 2, 44, 744]
19 [12, 1, 2137, 168, 96, 547, 34, 1, 103, 589, 2138, 52, 59, 1763, 7, 4280, 17, 615, 2]
10 [2798, 39, 149, 26, 823, 13, 2, 926, 2406, 2799]
6 [23, 12, 46, 3323, 2800, 16]
16 [2139, 36, 1945, 47, 4, 860, 20, 69, 2, 174, 548, 6, 1, 58, 275, 335]
11 [31, 1208, 53, 4281, 453, 22, 54, 146, 1, 2, 21]
14 [671, 9, 528, 305, 169, 55, 2407, 12, 2140, 1327, 182, 13, 47, 2]
14 [306, 372, 1, 4282, 6, 893, 14, 92, 466, 270, 2141, 5, 271, 2]
14 [2801, 266, 467, 966, 14, 453, 22, 107, 3324, 3325, 12, 17, 2, 475]
15 [3326, 616, 52, 186, 132, 175, 10

12 [430, 196, 487, 38, 7, 1451, 358, 4686, 220, 7, 77, 2]
20 [2947, 736, 548, 22, 1634, 213, 1, 1233, 6, 1, 608, 45, 42, 24, 525, 1, 1382, 6, 2, 928]
10 [4687, 1457, 1588, 27, 1, 238, 51, 2, 39, 58]
17 [39, 149, 2, 823, 539, 30, 882, 846, 22, 1, 295, 4688, 108, 117, 1, 42, 5]
17 [15, 59, 60, 1840, 60, 1383, 8, 60, 2548, 8, 220, 695, 6, 13, 1339, 2, 15]
16 [2948, 525, 582, 54, 990, 2475, 254, 3524, 1, 990, 62, 2949, 1118, 246, 376, 171]
10 [2950, 9, 38, 1, 936, 6, 16, 134, 1237, 2]
19 [2, 1, 948, 21, 12, 27, 1, 243, 104, 22, 4689, 187, 94, 28, 253, 2549, 1971, 213, 23]
21 [301, 943, 129, 39, 58, 29, 1, 2, 549, 458, 46, 635, 695, 4, 28, 11, 55, 7, 77, 126, 663]
7 [1558, 4690, 15, 107, 8, 1, 2]
12 [88, 5, 19, 175, 4, 11, 54, 22, 210, 211, 91, 2]
19 [296, 56, 105, 189, 4, 179, 20, 99, 131, 1785, 1190, 148, 4, 1298, 1136, 183, 14, 92, 104]
16 [671, 53, 88, 5, 2156, 4676, 134, 143, 4, 8, 75, 7, 502, 1299, 232, 2]
18 [1216, 53, 285, 1269, 6, 2550, 2, 137, 5, 180, 188, 80, 1474, 69, 1282, 46, 

13 [222, 218, 4, 65, 7, 1110, 954, 69, 6, 300, 1280, 2009, 2]
21 [1066, 1955, 4, 141, 204, 9, 111, 60, 468, 126, 72, 34, 1130, 1, 276, 141, 204, 1130, 15, 51, 2]
10 [1167, 1348, 2, 146, 25, 14, 197, 87, 275, 752]
18 [18, 25, 565, 34, 910, 1868, 27, 1049, 62, 61, 2274, 6, 87, 8, 15, 8, 96, 2]
20 [17, 1, 1381, 978, 6, 39, 149, 2, 46, 1, 453, 283, 4, 34, 930, 18, 25, 14, 930, 15]
15 [53, 3664, 131, 696, 75, 7, 2275, 69, 6, 295, 746, 1829, 22, 1090, 2]
11 [4900, 15, 347, 4, 1243, 2056, 38, 1, 295, 824, 2]
18 [40, 64, 2276, 22, 223, 14, 33, 138, 193, 166, 22, 505, 1472, 14, 7, 703, 1420, 2]
11 [4901, 96, 9, 43, 133, 1168, 9, 36, 133, 1222, 2]
12 [1414, 1415, 20, 21, 19, 29, 46, 939, 22, 29, 1, 2]
15 [10, 2609, 548, 18, 25, 507, 1, 26, 46, 1, 35, 28, 67, 1840, 2]
14 [3611, 443, 2156, 335, 59, 66, 6, 71, 592, 6, 2, 868, 4902, 714]
17 [1453, 38, 4903, 58, 1081, 6, 210, 416, 1109, 33, 352, 4904, 772, 9, 418, 33, 4350]
18 [1432, 2, 401, 266, 44, 4, 329, 26, 45, 7, 2277, 851, 104, 21, 1214, 24, 8

10 [1582, 59, 1, 123, 791, 43, 2940, 76, 2666, 2]
20 [358, 1779, 1, 607, 825, 29, 39, 149, 2, 2858, 253, 53, 1336, 5, 7, 759, 575, 24, 4, 53]
11 [1, 498, 1590, 5, 247, 981, 62, 3091, 3769, 1718, 2]
14 [1261, 23, 59, 1849, 293, 6, 2, 145, 121, 443, 44, 2024, 371, 265]
4 [3770, 23, 266, 2]
14 [99, 1023, 12, 8, 267, 60, 493, 6, 112, 1892, 79, 2, 401, 51]
21 [1, 174, 276, 4, 3592, 6, 7, 1784, 5, 4, 3771, 4, 13, 55, 1, 18, 25, 27, 211, 25, 2]
17 [120, 19, 1843, 4, 728, 1961, 578, 1708, 6, 3390, 767, 5, 60, 1277, 4, 325, 2]
22 [3053, 7, 1298, 246, 15, 121, 32, 59, 7, 3092, 1232, 199, 701, 549, 458, 3447, 24, 113, 1587, 4, 74, 521]
11 [853, 1500, 8, 3093, 107, 103, 53, 424, 3772, 45, 2271]
17 [68, 289, 81, 19, 1608, 4, 2619, 914, 7, 238, 1596, 4, 443, 44, 8, 2158, 2]
20 [1425, 1, 174, 108, 6, 1, 26, 121, 119, 46, 1, 3773, 160, 1115, 14, 1, 1700, 3094, 467, 1704]
19 [1, 103, 276, 1, 2, 287, 28, 253, 3095, 5, 56, 3096, 47, 8, 3774, 44, 137, 461, 1660]
8 [509, 3512, 197, 87, 5, 47, 45, 2]
15 [21

16 [93, 777, 34, 53, 288, 177, 14, 79, 32, 483, 210, 80, 499, 39, 58, 2]
11 [51, 296, 102, 1, 2, 13, 487, 16, 88, 59, 11]
13 [103, 7, 1244, 72, 566, 141, 204, 99, 17, 2327, 2, 154, 475]
22 [83, 116, 232, 387, 341, 32, 21, 2474, 55, 150, 3432, 292, 32, 21, 12, 6, 16, 833, 478, 17, 1719, 2]
23 [589, 46, 16, 166, 1024, 621, 24, 19, 308, 17, 48, 4, 65, 303, 3874, 973, 355, 900, 1, 486, 75, 2537, 2]
12 [7, 1094, 27, 39, 149, 2, 1580, 241, 110, 1891, 12, 1]
19 [1, 2, 39, 58, 21, 539, 9, 2632, 14, 8, 67, 1, 1446, 82, 34, 942, 14, 2927, 872]
20 [13, 565, 34, 1042, 1292, 45, 1, 2924, 4587, 4588, 29, 1, 2, 90, 368, 89, 4589, 19, 790, 700]
18 [15, 3123, 283, 1857, 921, 460, 9, 641, 83, 314, 3862, 411, 2326, 94, 75, 198, 13, 2]
9 [1162, 1494, 141, 204, 1109, 17, 420, 2569, 2]
11 [1, 628, 317, 1559, 150, 6, 40, 37, 132, 845, 2]
12 [3665, 3, 395, 4746, 623, 4747, 76, 46, 42, 507, 206, 3557]
20 [36, 7, 880, 282, 19, 627, 12, 1, 2, 429, 228, 431, 399, 610, 847, 505, 2471, 1560, 1362, 9]
14 [1427, 20, 

...

20 [509, 20, 26, 9, 111, 352, 67, 8, 778, 485, 22, 90, 35, 115, 101, 9, 85, 20, 506, 3]
4 [286, 5, 69, 3]
19 [4236, 9, 38, 301, 7, 426, 260, 33, 754, 32, 186, 67, 793, 109, 4, 2149, 71, 1459, 3]
17 [20, 5, 18, 25, 30, 846, 436, 221, 16, 40, 264, 38, 1879, 70, 310, 49, 3]
13 [366, 5, 1, 6, 46, 6, 150, 225, 4, 112, 379, 1697, 3]
19 [10, 15, 59, 148, 13, 123, 496, 72, 166, 22, 273, 19, 794, 34, 473, 13, 47, 2, 3]
19 [10, 83, 110, 73, 98, 86, 1, 114, 256, 22, 61, 250, 8, 135, 61, 147, 201, 27, 2]
19 [10, 5, 13, 36, 31, 70, 116, 120, 155, 84, 4, 202, 73, 98, 12, 1, 125, 2, 3]
19 [10, 15, 59, 148, 13, 123, 496, 72, 166, 22, 273, 19, 794, 34, 473, 13, 47, 2, 3]
17 [3, 91, 557, 4234, 38, 759, 648, 2760, 591, 3251, 40, 2362, 2080, 105, 4235, 2567, 249]
7 [333, 5, 7, 1207, 596, 906, 3]
4 [68, 1, 224, 3]
19 [10, 5, 13, 36, 31, 70, 116, 120, 155, 84, 4, 202, 73, 98, 12, 1, 125, 2, 3]
19 [20, 26, 9, 111, 352, 67, 8, 778, 485, 22, 90, 35, 115, 101, 9, 85, 20, 506, 3]
18 [9, 38, 301, 7, 426, 260, 33,

8 [1056, 42, 1205, 1408, 19, 310, 848, 3]
20 [10, 1, 35, 43, 28, 4, 152, 23, 18, 5, 161, 13, 14, 95, 62, 227, 74, 27, 175, 2]
17 [10, 127, 8, 23, 89, 4, 160, 293, 6, 20, 26, 40, 64, 65, 13, 2, 3]
19 [10, 83, 110, 73, 98, 86, 1, 114, 256, 22, 61, 250, 8, 135, 61, 147, 201, 27, 2]
19 [10, 83, 110, 73, 98, 86, 1, 114, 256, 22, 61, 250, 8, 135, 61, 147, 201, 27, 2]
21 [352, 38, 7, 494, 6, 173, 593, 48, 40, 451, 22, 7, 2303, 1110, 8, 167, 1027, 14, 16, 1110, 3]
20 [10, 1, 35, 43, 28, 4, 152, 23, 18, 5, 161, 13, 14, 95, 62, 227, 74, 27, 175, 2]
19 [10, 83, 110, 73, 98, 86, 1, 114, 256, 22, 61, 250, 8, 135, 61, 147, 201, 27, 2]
19 [10, 83, 110, 73, 98, 86, 1, 114, 256, 22, 61, 250, 8, 135, 61, 147, 201, 27, 2]
21 [10, 18, 25, 264, 348, 52, 1, 77, 340, 5, 123, 115, 9, 236, 23, 103, 59, 7, 440, 353, 320]
15 [10, 11, 72, 151, 111, 23, 5, 1, 465, 27, 119, 20, 26, 2, 3]
21 [159, 245, 428, 261, 235, 42, 515, 14, 52, 64, 293, 364, 388, 24, 587, 1, 132, 545, 17, 600, 546]
21 [159, 245, 428, 261, 235,

5 [14, 92, 377, 908, 3]
16 [10, 18, 25, 5, 165, 124, 4, 170, 1, 178, 6, 1, 77, 156, 2, 3]
16 [10, 18, 25, 5, 165, 124, 4, 170, 1, 178, 6, 1, 77, 156, 2, 3]
6 [4252, 513, 14, 1, 258, 3]
16 [10, 18, 25, 5, 165, 124, 4, 170, 1, 178, 6, 1, 77, 156, 2, 3]
16 [10, 18, 25, 5, 165, 124, 4, 170, 1, 178, 6, 1, 77, 156, 2, 3]
17 [1057, 786, 252, 1103, 17, 801, 37, 215, 101, 132, 499, 199, 297, 12, 20, 26, 3]
10 [9, 130, 1, 1, 391, 4068, 1, 457, 2266, 3]
5 [335, 380, 9, 559, 3]
10 [9, 1839, 66, 6, 11, 368, 4, 408, 2951, 3]
16 [10, 18, 25, 5, 165, 124, 4, 170, 1, 178, 6, 1, 77, 156, 2, 3]
16 [10, 18, 25, 5, 165, 124, 4, 170, 1, 178, 6, 1, 77, 156, 2, 3]
22 [159, 64, 257, 212, 31, 83, 168, 96, 5, 1285, 1234, 14, 49, 52, 82, 1127, 312, 74, 274, 560, 214, 3]
13 [10, 20, 3, 5, 41, 865, 357, 15, 8, 1030, 81, 19, 582]
19 [157, 305, 169, 43, 65, 747, 45, 408, 36, 1, 1942, 624, 878, 276, 4, 134, 1407, 169, 3]
17 [10, 50, 57, 153, 31, 6, 164, 162, 11, 130, 13, 33, 11, 43, 143, 2, 3]
16 [10, 18, 25, 5, 165, 

17 [10, 50, 57, 153, 31, 6, 164, 162, 11, 130, 13, 33, 11, 43, 143, 2, 3]
14 [9, 287, 151, 360, 14, 79, 53, 967, 21, 7, 110, 45, 442, 3]
25 [10, 9, 76, 87, 8, 15, 89, 4, 239, 231, 8, 240, 20, 172, 47, 129, 8, 75, 209, 6, 57, 8, 93, 2, 171]
17 [10, 50, 57, 153, 31, 6, 164, 162, 11, 130, 13, 33, 11, 43, 143, 2, 3]
17 [10, 50, 57, 153, 31, 6, 164, 162, 11, 130, 13, 33, 11, 43, 143, 2, 3]
10 [963, 292, 173, 17, 145, 593, 66, 294, 328, 3]
17 [20, 6, 168, 96, 5, 36, 66, 246, 83, 110, 471, 8, 36, 54, 1648, 4407, 521]
14 [56, 415, 72, 384, 1, 24, 1019, 435, 269, 48, 40, 661, 69, 3]
18 [68, 1, 103, 358, 17, 20, 145, 52, 5, 1448, 2087, 8, 81, 19, 7, 346, 2087, 3]
12 [83, 85, 9, 1565, 469, 27, 90, 49, 249, 53, 2145, 3]
17 [10, 50, 57, 153, 31, 6, 164, 162, 11, 130, 13, 33, 11, 43, 143, 2, 3]
6 [2036, 17, 27, 1, 3, 4]
6 [516, 15, 59, 517, 351, 3]
10 [245, 380, 373, 17, 261, 579, 871, 1096, 4428, 3]
14 [10, 18, 5, 262, 251, 1, 35, 50, 176, 1, 242, 91, 3, 3]
6 [448, 11, 1147, 805, 196, 3]
21 [10, 18

22 [4215, 56, 343, 1746, 4216, 47, 1, 1517, 203, 62, 1, 535, 6, 20, 26, 32, 2332, 4, 1235, 69, 3, 1758]
21 [159, 463, 212, 11, 1, 66, 139, 271, 22, 3, 35, 37, 1626, 198, 709, 4, 1518, 219, 114, 12, 727]
16 [400, 621, 1569, 1, 2, 21, 19, 2891, 1, 16, 287, 103, 478, 27, 3150, 335]
7 [42, 1205, 1408, 19, 310, 848, 3]
12 [20, 5, 83, 9, 43, 200, 18, 25, 454, 46, 1730, 3]
23 [296, 501, 69, 6, 1, 38, 7, 398, 340, 9, 182, 330, 4, 1144, 67, 22, 71, 1946, 14, 7, 215, 142, 2]
14 [174, 548, 6, 2, 21, 273, 568, 3097, 5, 1966, 21, 16, 654, 1385]
21 [49, 52, 144, 40, 37, 621, 37, 1906, 2750, 40, 111, 16, 7, 2651, 5, 7, 558, 6, 49, 123, 3]
18 [10, 83, 94, 23, 842, 187, 4, 1, 42, 63, 1, 787, 1004, 69, 4, 194, 2, 3]
19 [10, 15, 59, 148, 13, 123, 496, 72, 166, 22, 273, 19, 794, 34, 473, 13, 47, 2, 3]
23 [41, 1013, 1764, 64, 781, 14, 11, 118, 33, 11, 138, 2103, 1, 797, 4, 179, 95, 2786, 55, 757, 1604, 80, 3]
19 [10, 5, 13, 36, 31, 70, 116, 120, 155, 84, 4, 202, 73, 98, 12, 1, 125, 2, 3]
14 [10, 18, 5, 262

In [10]:
maxx = 0
c=0
for ll in X:
    c=len(ll)
    if c>maxx:
        maxx=c
print('Max',maxx)
X = pad_sequences(X)
print(len(X[0]))
print (X)

Max 29
29
[[   0    0    0 ... 1324 1409  743]
 [   0    0    0 ...  233  724   17]
 [   0    0    0 ...  207  371  670]
 ...
 [   0    0    0 ...   72   65    3]
 [   0    0    0 ... 1022 1423   74]
 [   0    0    0 ...  197    3  723]]


In [11]:
embed_dim = 128
lstm_out = 128
model = Sequential()#call the sequential model
model.add(Embedding(batch_size, embed_dim,input_length = X.shape[1], dropout=0.2))

model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))#
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

c:\users\mahalakshumi\myenv\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
c:\users\mahalakshumi\myenv\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 29, 128)           640000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 771,842
Trainable params: 771,842
Non-trainable params: 0
_________________________________________________________________
None


In [12]:

print(data['sentiment'].values)
Y = pd.get_dummies(data['sentiment']).values
print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

['Positive' 'Positive' 'Positive' ... 'Positive' 'Negative' 'Positive']
[[0 1]
 [0 1]
 [0 1]
 ...
 [0 1]
 [1 0]
 [0 1]]
(7188, 29) (7188, 2)
(3541, 29) (3541, 2)


In [25]:
model.fit(X_train, Y_train, nb_epoch = 3, batch_size=32, verbose = 2)

c:\users\mahalakshumi\myenv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/3
 - 11s - loss: 0.1219 - acc: 0.9516
Epoch 2/3
 - 11s - loss: 0.1050 - acc: 0.9598
Epoch 3/3
 - 11s - loss: 0.0885 - acc: 0.9642


In [40]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json

In [41]:
ckey=" "
csecret=" "
atoken=" "
asecret=" "

In [42]:
class listener(StreamListener):

    def on_data(self, data):
        all_data = json.loads(data)
        
        tweet = all_data["text"]
        s2 = tweet
        hh = tokenizer.texts_to_sequences([s2])
        X_test[9].shape
        hh = pad_sequences(hh, maxlen=29)
        hh=np.array(hh[0]).reshape(1,29)
        x=model.predict(hh)
        print(x)
        if(x[0][0]>x[0][1]):
            sentiment_value='negative'
        else:
            sentiment_value='positive'
        fp=open("sentiment_text.txt","a")
        fp.write(sentiment_value)
        fp.write('\n')
        fp.close()
        print((tweet,sentiment_value))
        
        return True

    def on_error(self, status):
        print (status)

In [ ]:
fp=open("sentiment_text.txt","w")
fp.close()
liveword=input("Enter the topic:")

In [66]:
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=[liveword])

[[0.81032723 0.18967274]]
('@madhukishwar @narendramodi Modi chose a yardstick to measure Nehru and erstwhile Cong Govt. and it showed them lac… https://t.co/2pXrlqufet', 'negative')
[[0.00479791 0.99520206]]
('RT @ashoswai: Raghuram Rajan was had a Ph.D. in Managment from MIT, Urjit Patel  MPhil in Economics from Oxford &amp; Ph.D. in Economics from Y…', 'positive')
[[0.40618786 0.5938122 ]]
('RT @nselrevenge: @NSELfight @ramgopaldass_77 @RavinarIN @narendramodi @imVkohli @shweta_shalini @SushmaSwaraj @rammadhavbjp @sanjeevsanyal…', 'positive')
[[0.40618786 0.5938122 ]]
('RT @ThunderIndia93: @sunilkpal @muralydoctrack1 @SunitaG1962 @RaiBahadur13 @Ashwita88 @romy1965 @kamleshtewari90 @balooni_satish @jyotsnava…', 'positive')
[[0.56599647 0.43400356]]
('RT @Gadhvilaxman: BC Arshad warsi ke fan bhi aa jao Modi aur BJP ko harane ka credit lelo aage nahi milne wala 😂😭 https://t.co/TboJossKqt', 'negative')
[[0.7013741  0.29862592]]
('RT @RoflGandhi_: Sangh ki head side ki kuttai ho gayi toh

[[0.9986078  0.00139225]]
('RT @_Abhinav96: One-way communication like “Mann ki Baat” where Modi plays Sant Baparam cannot be the only method of communication...😂😂\n\nMe…', 'negative')
[[0.8386374  0.16136262]]
('RT @AtomicBlow: Modi carved another neech for himself.\n#CongressWinsBig\nhttps://t.co/WklcBmDhyk', 'negative')
[[9.999964e-01 3.571101e-06]]
('RT @RavinarIN: Ten Messages For Modi... HORRIFYING to find "Justice 4 all, Appeasement of none" has turned into blazing Muslim-appeasement.…', 'negative')
[[9.9966693e-01 3.3310283e-04]]
('RT @BJP4India: It is only partnerships, that will get us to our goals - partnerships between citizens, partnerships between communities and…', 'negative')
[[0.99429375 0.00570617]]
('Market Bhakths! Adhia, Shaktikanta Das, Arun Jaitley, Urjit Patel &amp; everyone who works against their profit is wron… https://t.co/GHXunlSRfP', 'negative')
[[0.9864113  0.01358872]]
('RT @redLotus1_: Hunar haath, padho Pardes, nayi manzil, guraan-labdob, nayi roshni, 

KeyboardInterrupt: 